# Boolean Query

In this notebook, we will implement a Boolean information retrieval model by Elasticsearch.  
We have these steps: <br>

**1- Connect to the Elasticsearch Cluster and Create an Index <br>
2- Indexing Documents <br>
3- Boolean Retrieval (Fill where ever it says #TODO in this part)** <br>

Let me know if you have any problems with this notebook or implementation. <br>
**Telegram**: [@Mohammad_Ardestani4](https://t.me/Mohammad_Ardestani4) <br>
**Email**: mjavad.ardestani00@gmial.com

In [1]:
!pip install elasticsearch==7.13.0
!pip install tqdm

In [2]:
from elasticsearch import Elasticsearch, helpers
import os, json, time
from datetime import datetime
from tqdm import tqdm 

## Ignore The Security Warnings
Here we ignore the security warnings. It's suggested that you don't run below cell until you ensure that all warnings are about security. 


In [3]:
import warnings
warnings.filterwarnings('ignore')

## Load Processed Data

In [4]:
import json
dataset_path = 'IR_data_news_12k.json'
with open(dataset_path) as f:
    data = json.load(f)
print(data['0'].keys())

dict_keys(['title', 'content', 'tags', 'date', 'url', 'category'])


## Connect to the Elasticsearch Cluster and Create an Index
After starting your Elasticsearch on your pc (localhost:9200 is the default), we have to connect to it via the following piece of code.


In [5]:
index_name = 'my_elastic_index'
es = Elasticsearch("http://localhost:9200")

# Delete index if one does exist
if es.indices.exists(index=index_name):
    es.indices.delete(index=index_name)

es.indices.create(index = index_name)

{'acknowledged': True,
 'shards_acknowledged': True,
 'index': 'my_elastic_index'}

### Check the Cluster Status

In [6]:
dict(es.info())

{'name': 'DESKTOP-9SJ4G2F',
 'cluster_name': 'elasticsearch',
 'cluster_uuid': 'MXj13B4xRqC82nDZvTKp8g',
 'version': {'number': '7.4.2',
  'build_flavor': 'unknown',
  'build_type': 'unknown',
  'build_hash': '2f90bbf7b93631e52bafb59b3b049cb44ec25e96',
  'build_date': '2019-10-28T20:40:44.881551Z',
  'build_snapshot': False,
  'lucene_version': '8.2.0',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

## Indexing Documents 

we can add our documents to created index in two ways:<br>
1- one by one in for loop. <br>
2- use [Bulk API](https://www.elastic.co/guide/en/elasticsearch/reference/current/docs-bulk.html#:~:text=Bulk%20APIedit,can%20greatly%20increase%20indexing%20speed.), which performs multiple indexing or delete operations in a single API call. This reduces overhead and can significantly increase indexing speed.

###  for loop by es.index 
**You dont have to run this cell**. 

In [7]:
# for i in tqdm(range(len(data))):
#     es.index(index = index_name, id=i, document=data[str(i)])
#     i = i + 1

###  Bulk API

In [8]:
from elasticsearch.helpers import bulk

def bulk_sync():
    actions = [
        {
            '_index': index_name,
            '_id': doc_id,
            '_source': doc
        } for doc_id,doc in data.items()
    ]
    bulk(es, actions)


In [9]:
start = time.time()
bulk_sync()
end = time.time()
print("Indexing all documents took about {:.2f} seconds".format(end - start))

Indexing all documents took about 31.43 seconds


### check index

In [10]:
es.count(index = index_name)

{'count': 12202,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}}

## Boolean Retrieval 

### Bool Query Structure

It is built using one or more boolean clauses, each clause with a typed occurrence. The occurrence types are:

**must**: The clause (query) must appear in matching documents and will contribute to the score.
<br><br>
**filter**: The clause (query) must appear in matching documents. However unlike must the score of the query will be ignored. Filter clauses are executed in filter context, meaning that scoring is ignored and clauses are considered for caching.
<br><br>
**should**: The clause (query) should appear in the matching document.
<br><br>
**must_not**: The clause (query) must not appear in the matching documents. Clauses are executed in filter context meaning that scoring is ignored and clauses are considered for caching. Because scoring is ignored, a score of 0 for all documents is returned.

For further information, you can read this [Document](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-bool-query.html). 

###  <span style="color:red"> TODO </span>
You should read about [match query](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-match-query.html#match-top-level-params) and [match phrase query](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-match-query-phrase.html) then complete the below cell.

In [29]:
query_json = {
  "query": {
    "bool": {
      "should": [
        # {"match": {"content": {"query": "آمریکا"}}},
        # {"match_phrase": {"content": {"query": "تحریم هسته‌ای"}}},
      ],
      "must_not": [
        # {"match": {"content": {"query": "ایران"}}},
      ],
    },
  }
}

query = input('کوئری خود را وارد کنید: ')

# should -> match_phrases
quotations = [i for i in range(len(query)) if query[i] == "\""]
if len(quotations)%2 == 1: quotations.pop() # odd number of quotations

for i in range(0, len(quotations), 2): # save phrases in a list
    match_ph = (query[quotations[i]+1:quotations[i+1]])
    query_json['query']['bool']['should'].append({"match_phrase": {"content": {"query": match_ph}}})

# must_not, should -> match
from parsivar import Tokenizer, Normalizer
my_normalizer = Normalizer()
my_tokenizer = Tokenizer()

query_copy = query
query_copy = "\"" + query_copy + "\""
quotations_copy = [i for i in range(len(query_copy)) if query_copy[i] == "\""]
for i in range(0, len(quotations_copy), 2):
      x = (query_copy[quotations_copy[i]+1:quotations_copy[i+1]])
      x1 = my_normalizer.normalize(x)
      x2 = my_tokenizer.tokenize_words(x1)
      j = 0
      while j < len(x2):
          if x2[j] == "!":
            match = x2[j+1]
            query_json['query']['bool']['must_not'].append({"match": {"content": {"query": match}}})
            j += 2
          else:
            query_json['query']['bool']['should'].append({"match": {"content": {"query": x2[j]}}})
            j += 1

کوئری خود را وارد کنید: "تحریم هسته‌ای" آمریکا ! ایران


In [30]:
print(query)
print(query_json)

"تحریم هسته‌ای" آمریکا ! ایران
{'query': {'bool': {'should': [{'match_phrase': {'content': {'query': 'تحریم هسته\u200cای'}}}, {'match': {'content': {'query': 'آمریکا'}}}], 'must_not': [{'match': {'content': {'query': 'ایران'}}}]}}}


### Search query

The _source field contains the original JSON document body that was passed at index time. The _source field itself is not indexed (and thus is not searchable), but it is stored so that it can be returned when executing fetch requests, like get or search.

For further information, you can read this [Document](https://www.elastic.co/guide/en/elasticsearch/reference/current/search-search.html#search-api-response-body).

In [32]:
start = time.time()
res = es.search(index=index_name, body=query_json, _source= ["url"])
res = dict(res)
end = time.time()
print("searching documents took about {:.2f} seconds".format(end - start))

searching documents took about 3.33 seconds


### Results

In [33]:
print("{} results in {} s: ".format(res['hits']['total']['value'] ,res['took']/1000))
for doc in res['hits']['hits']:
    print(doc['_source']['url'])

171 results in 0.882 s: 
https://www.farsnews.ir/news/14001214000789/نماینده-کلیمیان-در-مجلس-دهم--کارتل‌ها-بر-تصمیمات-هیأت-حاکمه-آمریکا
https://www.farsnews.ir/news/14001217000665/خباثت‌های-آمریکا-در-برجام-روسیه-و-چین-را-هم-به-این-کشور-بدبین-کرد
https://www.farsnews.ir/news/14001213000089/آمریکا-با-بحران‌آفرینی-به-حیات-خود-ادامه-می-دهد
https://www.farsnews.ir/news/14001211000321/آمریکا-دولت‌های-متحد-خود-را-در-زمان-اضطرار-تنها-می‌گذارد
https://www.farsnews.ir/news/14001211000898/سود-مافیای-اسلحه‌سازی-آمریکا-در-ناامن-بودن-جهان-است
https://www.farsnews.ir/news/14001214000825/آمریکا-رژیمی-مافیایی-است-و-مردم-در-تصمیمات-حاکمان-آن-جایگاهی-ندارند
https://www.farsnews.ir/news/14001212000725/آمریکا-برای-فروش-تسلیحات-خود-به-ایجادناامنی-و-بحران‌آفرینی-نیاز-دارد
https://www.farsnews.ir/news/14001204000444/۲۸-آمریکایی-در-جام-یاشاردوغو-جردن-باروز-هم-می‌آید
https://www.farsnews.ir/news/14001213000328/کارتل‌های-اقتصادی-رؤسای-جمهور-آمریکا-را-تعیین-می‌کنند
https://www.farsnews.ir/news/14001211000220/نمای